<a href="https://colab.research.google.com/github/phuneil1001/My_Profile/blob/main/train_micro_speech_model_phuneil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>